In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import nltk
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model

# **Loading data from Github repo**

In [2]:
names = ['id',	'name',	'host_id',	'host_name',	'neighbourhood_group',	'neighbourhood',	'latitude',	
         'longitude',	'room_type',	'price',	'minimum_nights',	'number_of_reviews',	'last_review',	
         'reviews_per_month',	'calculated_host_listings_count',	'availability_365']
df = pd.read_csv('https://raw.githubusercontent.com/lmxy0212/ML_project/master/new-york-city-airbnb-open-data/AB_NYC_2019.csv',
                names=names,na_values='?',header=None)
df = df.dropna()
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,6,7,9,10,11,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
1,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
2,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
4,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
5,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


# **Formating data to be np array**

In [3]:
airbnb_name = np.array(df['name'][1:])
reviews_per_month = np.array(df['reviews_per_month'][1:])
n = np.shape(airbnb_name)[0]
print(n)
print(airbnb_name)
print(np.array(reviews_per_month))
print(np.shape(airbnb_name))
print(np.shape(reviews_per_month))


38821
['Clean & quiet apt home by the park' 'Skylit Midtown Castle'
 'Cozy Entire Floor of Brownstone' ... 'Seas The Moment'
 '1B-1B apartment near by Metro' 'Cozy Private Room in Bushwick, Brooklyn']
['0.21' '0.38' '4.64' ... 1.0 2.0 1.0]
(38821,)
(38821,)


# **Creating bag of words for each airbnb name**

In [4]:
vectorizer = CountVectorizer()
bow = vectorizer.fit(airbnb_name)
bow = vectorizer.transform(airbnb_name)
bow

<38821x6961 sparse matrix of type '<class 'numpy.int64'>'
	with 222889 stored elements in Compressed Sparse Row format>

In [0]:
# print(bow)
# # (seq_num, feature_num)  count
# print(vectorizer.vocabulary_.get("great")) #feature number of "great"

In [6]:
X = bow.toarray()
print(X)
print(np.shape(X))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(38821, 6961)


# **Train Test Split**

In [7]:
Xtr, Xts, ytr, yts = train_test_split(X, reviews_per_month, test_size = 1/3)
print("Xtr:\n", Xtr, "\nXts:\n",Xts,"\nytr:\n", ytr, "\nyts:\n", yts)
print("\nXtr.shape:", Xtr.shape, "Xts.shape", Xts.shape)
print("\nytr.shape:", ytr.shape, "yts.shape:", yts.shape)
ytr = ytr.astype(np.float)
yts = yts.astype(np.float)

Xtr:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
Xts:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
ytr:
 ['0.17' '0.10' '0.05' ... '0.08' '0.12' 1.28] 
yts:
 ['3.47' '1.79' '4.17' ... 1.01 '1.74' '3.33']

Xtr.shape: (25880, 6961) Xts.shape (12941, 6961)

ytr.shape: (25880,) yts.shape: (12941,)


In [0]:
def plot_y_and_ypred(t,y,y_pred, y1, y2):
  """
    plot y and ypred again t
    y1, y2 are strings
  """
  plt.plot(t, y,'.')
  plt.plot(t, y_pred)
  plt.legend([y1,y2])
  plt.grid()
  plt.xlabel(y1, fontsize=16)
  plt.ylabel(y2, fontsize=16)

# **Using linear regression to fit the data and saving in linear_reg.p**

In [9]:
regr = linear_model.LinearRegression()
regr.fit(Xtr,ytr)
print(regr.intercept_)
print(regr.coef_)


NameError: ignored

In [0]:
with open( "linear_reg.p", "wb" ) as fp:
    pickle.dump( [regr, Xtr, ytr, Xts, yts],  fp)

In [12]:
with open( "linear_reg.p", "rb" ) as fp:
    regr, Xtr, ytr, Xts, yts = pickle.load(fp)
ytr_pred = regr.predict(Xtr)
lossm = np.linalg.norm(ytr_pred - ytr)**2
lossm = (1/n) * np.linalg.norm(ytr_pred - ytr)**2
print("Average loss on training data:",lossm)
yts_pred = regr.predict(Xts)
lossm = (1/n) * np.linalg.norm(yts_pred - yts)**2
print("Average loss on test data:", lossm)


Average loss on training data: 1.2994240550299205
Average loss on test data: 9.864150215880753e+23
